In [1]:
import re
import pdfplumber
import pandas as pd

In [2]:
# Load the PDF file
pdf_path = "./CoC APRs from HMIS/143 CoC APR for CARF.pdf"

In [5]:
page_number = 11

with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[page_number]

    # Define custom table settings
    table_settings = {
        "vertical_strategy": "lines",  # "lines", "text", or "explicit"
        "horizontal_strategy": "lines",  # "lines", "text", or "explicit"
        "intersection_tolerance": 5,
    }

    # Extract table using custom settings
    tables = page.extract_tables(table_settings=table_settings)

    print(f"Found {len(tables)} tables on page {page_number + 1}")

    for idx, table in enumerate(tables):
        df = pd.DataFrame(table)
        print(f"\nTable {idx + 1}:")
        print(df)


Found 0 tables on page 12


In [3]:
# specify page number (0-based, i.e. if we want info on page 12, we should assign page_number to 11)
page_number = 11

# Print all the text on that page
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[page_number]
    text = page.extract_text()
    
    for line in text.split('\n'):
        print(line)

Adults with Missing Income Information 35 0 0
Number of adult stayers not yet required to have an annual assessment 102
Number of adult stayers without required annual assessment 110
TotalA dults 474 386 88
I
1 or More Source of Income 193 101 41
Adults with Income Information at Start and Annual AssessmenVExit 174 86
19a1 • ClientC ashI ncomeC hange• IncomeS ource• by Start and LatestS tatus
Income Change
Did Not Have
by Income Performance
Retained Retained the Income
Category Had Income Retained Did Not Have Measure:A dults Performance
Income Income Category at
(Universe: Adult Category at Income the Income who Gainedo r measure:
Category But Category and Start and Total Adults
Stayers with Start and Did Category and Category at Increased Percento f
Had Less $at Same $at Gained the (including those
Income Not Have It at Increased $ at Start or at Incomef rom Personsw ho
Annual Annual Income with No Income)
Information at Annual Annual Annual Start to Annual Accomplished
Assessment As

In [22]:
# Print all the text for section 19a1
# Split them in lines
lines = text.split('\n')

# Initialize indices
start_index = None
end_index = None

# Find start index for 19a1
for i, line in enumerate(lines):
    if re.search(r'19a1', line):
        start_index = i
        # print(f"Start found at line {i}: {line}")
        break

# Find end index for 19a2, searching AFTER start_index
if start_index is not None:
    for j in range(start_index + 1, len(lines)):
        if re.search(r'19a2', lines[j]):
            end_index = j
            # print(f"End found at line {j}: {lines[j]}")
            break

# Slice the section between 19a1 and 19a2
if start_index is not None and end_index is not None:
    sliced_lines = lines[start_index : end_index]
elif start_index is not None:
    # If there's no 19a2 found, go until the end
    sliced_lines = lines[start_index :]

# Print the extracted section
for line in sliced_lines:
    print(line)


19a1 • ClientC ashI ncomeC hange• IncomeS ource• by Start and LatestS tatus
Income Change
Did Not Have
by Income Performance
Retained Retained the Income
Category Had Income Retained Did Not Have Measure:A dults Performance
Income Income Category at
(Universe: Adult Category at Income the Income who Gainedo r measure:
Category But Category and Start and Total Adults
Stayers with Start and Did Category and Category at Increased Percento f
Had Less $at Same $at Gained the (including those
Income Not Have It at Increased $ at Start or at Incomef rom Personsw ho
Annual Annual Income with No Income)
Information at Annual Annual Annual Start to Annual Accomplished
Assessment Assessment as Category at
Start and Assessment Assessment Assessment Assessment, this Measure
Than at Start at Start Annual
Annual AverageG ain
Assessment
Assessment)
Number of
Adults with
Earned Income
(i.e., 11 0 0 8 12 143 174 20 11%
Employment
Income)
I
Average Change
in Earned -1113 989 1273 1160
Income
Number of
Ad

In [23]:
# Join all sliced lines into a single block of text
sliced_text = '\n'.join(sliced_lines)

# Combine sliced_lines into meaningful blocks before extracting numbers
combined_lines = []
temp_line = ""

for i, line in enumerate(sliced_lines):
    # Skip the first line if it contains '19a1' (or any unwanted header)
    if i == 0 and re.search(r'19a1', line):
        continue

    temp_line += " " + line.strip()

    # If the line contains numbers, add it to combined_lines and reset
    if re.search(r'\d', line):
        combined_lines.append(temp_line.strip())
        temp_line = ""  # Reset for the next block

# Now extract numbers from each combined line and print them out per line
for line in combined_lines:
    # Extract numbers, including percentages
    numbers = re.findall(r'-?\d+\.?\d*%?', line)
    
    if numbers:
        # Print all numbers in one line
        print(" ".join(numbers))

11 0 0 8 12 143 174 20 11%
-1113 989 1273 1160
5 12 4 45 25 83 174 70 40%
759 -391 302 1049 569
10 14 3 55 30 62 174 85 49%
-926 -503 461 1135 247 699


In [ ]:
# I am thinking about we can pre-defined the table structure i.e. row and col names, and fill in the numbers